In [1]:
import pickle
import numpy as np
import tensorflow as tf

# Load pickled data
with open('small_traffic_set/small_train_traffic.p', mode='rb') as f:
    data = pickle.load(f)

In [2]:
# split data
X_train, y_train = data['features'], data['labels']

In [5]:
X_train.shape

(100, 32, 32, 3)

In [6]:
# Setup Keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten

Using TensorFlow backend.


In [8]:
# Set the first layer to a Flatten() layer with the input_shape set to (32, 32, 3).
# Set the second layer to a Dense() layer with an output width of 128.
# Use a ReLU activation function after the second layer.
# Set the output layer width to 5, because for this data set there are only 5 classes.
# Use a softmax activation function after the output layer.
# Train the model for 3 epochs. You should be able to get over 50% training accuracy

In [16]:
# TODO: Build the Fully Connected Neural Network in Keras Here
model = Sequential()
model.add(Flatten(input_shape=(32, 32, 3)))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(5))
model.add(Activation('softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               393344    
_________________________________________________________________
activation_6 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 645       
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
Total params: 393,989
Trainable params: 393,989
Non-trainable params: 0
_________________________________________________________________


In [17]:
# preprocess data
X_normalized = np.array(X_train / 255.0 - 0.5 )

from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)

model.compile('adam', 'categorical_crossentropy', ['accuracy'])
# TODO: change the number of training epochs to 3
history = model.fit(X_normalized, y_one_hot, epochs=3, validation_split=0.2)

/root/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/sklearn/utils/fixes.py:313: FutureWarning: numpy not_equal will not check object identity in the future. The comparison did not return the same result as suggested by the identity (`is`)) and will change.
  _nan_object_mask = _nan_object_array != _nan_object_array


Train on 80 samples, validate on 20 samples
Epoch 1/3
80/80 [==============================] - 1s 7ms/step - loss: 1.5391 - acc: 0.4250 - val_loss: 0.8085 - val_acc: 0.6000
Epoch 2/3
80/80 [==============================] - 0s 236us/step - loss: 0.9297 - acc: 0.6375 - val_loss: 0.5509 - val_acc: 0.9500
Epoch 3/3
80/80 [==============================] - 0s 199us/step - loss: 0.6719 - acc: 0.7375 - val_loss: 0.5205 - val_acc: 0.7500


In [25]:
import os
import sys
import cv2
import json
import numpy as np
import keras
from keras.layers.core import Dense, Activation, Flatten
from keras.activations import relu, softmax
    
def get_result(model, history):
    
    result = {'is_correct': False, 'error': False, 'values': [],
              'output': '', 'feedback': '', 'comment': ""}
    
    try:       
        check_shape_expanded = None
        check_shape_mixed = None
        check_shape_compressed = None
        
        if len(model.layers) > 4:        
            check_shape_expanded = (model.layers[0].input_shape == (None, 32, 32, 3)) & \
                          (model.layers[1].output_shape == (None, 128)) & \
                          (model.layers[2].activation == relu) & \
                          (model.layers[3].output_shape == (None, 5)) & \
                          (model.layers[4].activation == softmax)
        elif len(model.layers) > 3:
            check_shape_mixed = (model.layers[0].input_shape == (None, 32, 32, 3)) & \
                          (model.layers[1].output_shape == (None, 128)) & \
                          (model.layers[1].activation == keras.activations.relu) & \
                          (model.layers[2].output_shape == (None, 5)) & \
                          (model.layers[3].activation == softmax)
            if not check_shape_mixed:
                check_shape_mixed = (model.layers[0].input_shape == (None, 32, 32, 3)) & \
                          (model.layers[1].output_shape == (None, 128)) & \
                          (model.layers[2].activation == relu) & \
                          (model.layers[3].output_shape == (None, 5)) & \
                          (model.layers[3].activation == keras.activations.softmax)
        else:        
            check_shape_compressed = (model.layers[0].input_shape == (None, 32, 32, 3)) & \
                          (model.layers[1].output_shape == (None, 128)) & \
                          (model.layers[1].activation == keras.activations.relu) & \
                          (model.layers[2].output_shape == (None, 5)) & \
                          (model.layers[2].activation == keras.activations.softmax)
        

        if check_shape_expanded or check_shape_mixed or check_shape_compressed:
            if history.history['acc'][-1] > 0.5:
                result["is_correct"] = True
                result["comment"] = 'Looks good!'
                result["feedback"] = 'Nice work!'
            else:
                result["is_correct"] = False
                result["comment"] = 'The accuracy was less than 50%'
                result["feedback"] = 'Make sure you are running the model for enough epochs'
        else:
            result["is_correct"] = False
            result["comment"] = 'The model layout looks incorrect'
            result["feedback"] = 'Try following the model layout from the instructions'
    except Exception as err:
        result['is_correct'] = False
        result['feedback'] = 'Oops, looks like you got an error!'
        result['error'] = str(err)

    return result

def run_grader(model, history):
    
    try:
    # Get grade result information
        result = get_result(model, history)
    except Exception as err:
        # Default error result
        result = {
            'correct': False,
            'feedback': 'Something went wrong with your submission:',
            'comment': str(err)}

    feedback = result.get('feedback')
    comment = result.get('comment')

    print(feedback,comment)

In [26]:
try:
    run_grader(model, history)
except Exception as err:
    print(str(err))

Nice work! Looks good!
